# DC Economic Data - Monthly Dataset Creation
=================================================

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from scipy import interpolate

## 1. Create Monthly Date Range

In [30]:
start_date = '2000-01-01'
end_date = '2024-12-31'

# Create a date range for all months in the period
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
monthly_df = pd.DataFrame({'Date': date_range})
monthly_df['Year'] = monthly_df['Date'].dt.year
monthly_df['Month'] = monthly_df['Date'].dt.month

print(f"Created base monthly DataFrame with {len(monthly_df)} records from {monthly_df['Date'].min().strftime('%Y-%m-%d')} to {monthly_df['Date'].max().strftime('%Y-%m-%d')}")

Created base monthly DataFrame with 300 records from 2000-01-01 to 2024-12-01


## 2. Process Data Files with Monthly Data

### Process Unemployment Rate (already monthly)

In [31]:
try:
    unemployment_df = pd.read_csv(os.path.join('Data', 'DC_Unemployment_Rate.csv'))
    
    # Parse the Label column to extract year and month
    unemployment_df[['Year', 'Month']] = unemployment_df['Label'].str.split(' ', expand=True)
    
    # Map month names to numbers
    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
                 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    unemployment_df['Month'] = unemployment_df['Month'].map(month_map)
    
    # Convert Year to integer
    unemployment_df['Year'] = unemployment_df['Year'].astype(int)
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        unemployment_df[['Year', 'Month', 'Value']].rename(columns={'Value': 'Unemployment_Rate'}),
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added monthly Unemployment Rate data ({unemployment_df['Year'].min()}-{unemployment_df['Year'].max()})")
except Exception as e:
    print(f"Error processing Unemployment Rate data: {e}")

Added monthly Unemployment Rate data (1999-2024)


### Process CPI Data (monthly in wide format)

In [32]:
try:
    cpi_df = pd.read_csv(os.path.join('Data', 'DMV_CPI.csv'))
    
    # Convert from wide to long format
    cpi_long = pd.melt(
        cpi_df,
        id_vars=['Year'],
        value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
        var_name='Month',
        value_name='CPI'
    )
    
    # Map month names to numbers
    month_map = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
                 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    cpi_long['Month'] = cpi_long['Month'].map(month_map)
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        cpi_long[['Year', 'Month', 'CPI']],
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added monthly CPI data ({cpi_df['Year'].min()}-{cpi_df['Year'].max()})")
except Exception as e:
    print(f"Error processing CPI data: {e}")

Added monthly CPI data (1999-2024)


### Process Interest Rate Data (monthly)

In [33]:
try:
    interest_df = pd.read_csv(os.path.join('Data', 'Interest_Rates.csv'))
    
    # Extract year and month from observation_date
    interest_df['Date'] = pd.to_datetime(interest_df['observation_date'])
    interest_df['Year'] = interest_df['Date'].dt.year
    interest_df['Month'] = interest_df['Date'].dt.month
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        interest_df[['Year', 'Month', 'FEDFUNDS']].rename(columns={'FEDFUNDS': 'Interest_Rate'}),
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added monthly Interest Rate data ({interest_df['Year'].min()}-{interest_df['Year'].max()})")
except Exception as e:
    print(f"Error processing Interest Rate data: {e}")

Added monthly Interest Rate data (2000-2025)


### Process Mortgage Rate Data (weekly to monthly)

In [34]:
try:
    mortgage_df = pd.read_csv(os.path.join('Data', 'Mortgage_Rate.csv'))
    
    # Convert Week to datetime
    mortgage_df['Date'] = pd.to_datetime(mortgage_df['Week'], errors='coerce')
    
    # Filter out any rows with invalid dates
    mortgage_df = mortgage_df.dropna(subset=['Date'])
    
    # Extract year and month
    mortgage_df['Year'] = mortgage_df['Date'].dt.year
    mortgage_df['Month'] = mortgage_df['Date'].dt.month
    
    # Group by year and month to get monthly average
    mortgage_monthly = mortgage_df.groupby(['Year', 'Month'])['FRM'].mean().reset_index()
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        mortgage_monthly.rename(columns={'FRM': 'Mortgage_Rate'}),
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added monthly Mortgage Rate data ({mortgage_monthly['Year'].min()}-{mortgage_monthly['Year'].max()})")
except Exception as e:
    print(f"Error processing Mortgage Rate data: {e}")

Added monthly Mortgage Rate data (1999-2025)


### Process ZHVI (Zillow Home Value Index) - already in monthly format

In [35]:
try:
    zhvi_df = pd.read_csv(os.path.join('Data', 'zhvi_home_values.csv'))
    
    # Filter for Washington DC - as the file should only have DC data
    dc_data = zhvi_df[zhvi_df['RegionName'].str.contains('Washington', case=False)]
    
    if len(dc_data) == 0:
        # If no data matches the filter, use all data (should just be DC)
        dc_data = zhvi_df
    
    if len(dc_data) > 0:
        # Get date columns (excluding metadata columns)
        date_columns = [col for col in dc_data.columns if '/' in col]
        
        # Convert wide format to long format
        zhvi_long = pd.melt(
            dc_data,
            id_vars=['RegionID', 'RegionName'],
            value_vars=date_columns,
            var_name='Date',
            value_name='ZHVI'
        )
        
        # Convert Date to datetime
        zhvi_long['Date'] = pd.to_datetime(zhvi_long['Date'])
        
        # Extract year and month
        zhvi_long['Year'] = zhvi_long['Date'].dt.year
        zhvi_long['Month'] = zhvi_long['Date'].dt.month
        
        # Merge with monthly dataframe
        monthly_df = pd.merge(
            monthly_df,
            zhvi_long[['Year', 'Month', 'ZHVI']],
            on=['Year', 'Month'],
            how='left'
        )
        
        print(f"Added monthly ZHVI data ({zhvi_long['Year'].min()}-{zhvi_long['Year'].max()})")
    else:
        print("ZHVI data did not contain Washington DC data")
except Exception as e:
    print(f"Error processing ZHVI data: {e}")

Added monthly ZHVI data (2000-2024)


## 3. Process Annual Data with Monthly Interpolation

In [36]:
# Function to convert annual data to monthly with interpolation
def annual_to_monthly(annual_df, year_col, value_col, new_col_name, date_col=None):
    """
    Convert annual data to monthly with interpolation
    """
    # If a date column is provided, use it. Otherwise create one from the year
    if date_col is None:
        annual_df['Date'] = pd.to_datetime(annual_df[year_col].astype(str) + '-01-01')
    else:
        annual_df['Date'] = pd.to_datetime(annual_df[date_col])
    
    # Set the date as index
    annual_df = annual_df.set_index('Date')
    
    # Create a monthly date range based on the annual data's date range
    monthly_dates = pd.date_range(
        start=annual_df.index.min(),
        end=annual_df.index.max(),
        freq='MS'
    )
    
    # Create a new dataframe with monthly dates
    monthly_data = pd.DataFrame(index=monthly_dates)
    
    # Add the annual data to the monthly dataframe
    for date, row in annual_df.iterrows():
        if date in monthly_data.index:
            monthly_data.loc[date, value_col] = row[value_col]
    
    # Interpolate missing values
    monthly_data[value_col] = monthly_data[value_col].interpolate(method='cubic')
    
    # Reset index to get Date as a column
    monthly_data = monthly_data.reset_index()
    monthly_data = monthly_data.rename(columns={'index': 'Date', value_col: new_col_name})
    
    # Add Year and Month columns
    monthly_data['Year'] = monthly_data['Date'].dt.year
    monthly_data['Month'] = monthly_data['Date'].dt.month
    
    return monthly_data[['Year', 'Month', new_col_name]]

### Process GDP Growth Data (quarterly to monthly)

In [37]:
try:
    gdp_df = pd.read_csv(os.path.join('Data', 'GDP_Growth.csv'))
    
    # This data has a unique structure:
    # First row is years for each quarter
    # Second row is quarter labels (Q1, Q2, Q3, Q4)
    
    # Separate years and quarters from data
    years_quarters = {}  # To store year-quarter relationships
    years = []
    quarters = []
    gdp_values = []
    
    # Extract headers (column names) as years and quarters
    all_columns = gdp_df.columns.tolist()
    
    # For each column after the first row
    for i, col in enumerate(all_columns):
        try:
            # First row should have years (column names)
            # Second row should have quarter labels (Q1, Q2, etc.)
            year = int(float(col))
            quarter = gdp_df.iloc[0, i]  # Get the quarter from the first row
            
            # The value is in the second row
            value = float(gdp_df.iloc[1, i])
            
            years.append(year)
            quarters.append(quarter)
            gdp_values.append(value)
            
            # Store in our dictionary
            years_quarters[(year, quarter)] = value
        except (ValueError, TypeError):
            # Skip any column that doesn't convert to a year
            continue
    
    # Convert to a DataFrame
    gdp_quarterly = pd.DataFrame({
        'Year': years,
        'Quarter': quarters,
        'GDP_Growth': gdp_values
    })
    
    # Map each quarter to its middle month
    quarter_to_month = {'Q1': 2, 'Q2': 5, 'Q3': 8, 'Q4': 11}
    gdp_quarterly['Month'] = gdp_quarterly['Quarter'].map(quarter_to_month)
    
    # Create date column for interpolation
    gdp_quarterly['Date'] = pd.to_datetime(
        gdp_quarterly['Year'].astype(str) + '-' + 
        gdp_quarterly['Month'].astype(str) + '-01'
    )
    
    # Set date as index for interpolation
    gdp_quarterly = gdp_quarterly.set_index('Date')
    
    # Create a monthly date range
    monthly_dates = pd.date_range(
        start=gdp_quarterly.index.min(),
        end=gdp_quarterly.index.max(),
        freq='MS'
    )
    
    # Create monthly dataframe
    gdp_monthly = pd.DataFrame(index=monthly_dates)
    
    # Map quarterly values to their corresponding months
    for date, row in gdp_quarterly.iterrows():
        if date in gdp_monthly.index:
            gdp_monthly.loc[date, 'GDP_Growth'] = row['GDP_Growth']
    
    # Interpolate missing values
    gdp_monthly['GDP_Growth'] = gdp_monthly['GDP_Growth'].interpolate(method='cubic')
    
    # Prepare for merge
    gdp_monthly = gdp_monthly.reset_index()
    gdp_monthly['Year'] = gdp_monthly['index'].dt.year
    gdp_monthly['Month'] = gdp_monthly['index'].dt.month
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        gdp_monthly[['Year', 'Month', 'GDP_Growth']],
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added GDP Growth data (quarterly interpolated to monthly)")
except Exception as e:
    print(f"Error processing GDP Growth data: {e}")
    import traceback
    traceback.print_exc()

Added GDP Growth data (quarterly interpolated to monthly)


### Process Median Household Income (annual)

In [38]:
try:
    income_df = pd.read_csv(os.path.join('Data', 'DC_Median_Household_Income_Annually.csv'))
    
    # Convert to monthly
    income_monthly = annual_to_monthly(
        income_df,
        'Year',
        'MEHOINUSDCA646N',
        'Median_Household_Income',
        'observation_date'
    )
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        income_monthly,
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added Median Household Income data (interpolated to monthly)")
except Exception as e:
    print(f"Error processing Median Household Income data: {e}")
    # Extract year from observation_date and use it directly
    try:
        income_df = pd.read_csv(os.path.join('Data', 'DC_Median_Household_Income_Annually.csv'))
        income_df['Year'] = pd.to_datetime(income_df['observation_date']).dt.year
        
        # Create a temporary dataframe with annual data
        temp_df = pd.DataFrame()
        for year in income_df['Year'].unique():
            year_income = income_df[income_df['Year'] == year]['MEHOINUSDCA646N'].values[0]
            for month in range(1, 13):
                temp_df = pd.concat([temp_df, pd.DataFrame({
                    'Year': [year],
                    'Month': [month],
                    'Date': [pd.to_datetime(f"{year}-{month:02d}-01")],
                    'Median_Household_Income': [year_income]
                })])
        
        # Sort by date
        temp_df = temp_df.sort_values('Date')
        
        # Interpolate to smooth out the steps
        temp_df['Median_Household_Income'] = temp_df['Median_Household_Income'].interpolate(method='cubic')
        
        # Merge with monthly dataframe
        monthly_df = pd.merge(
            monthly_df,
            temp_df[['Year', 'Month', 'Median_Household_Income']],
            on=['Year', 'Month'],
            how='left'
        )
        
        print(f"Added Median Household Income data (alternative method)")
    except Exception as e2:
        print(f"Second attempt to process income data failed: {e2}")

Added Median Household Income data (interpolated to monthly)


### Process Population Data (annual)

In [39]:
try:
    population_df = pd.read_csv(os.path.join('Data', 'DC_Population.csv'))
    
    # Extract year from observation_date
    population_df['Year'] = pd.to_datetime(population_df['observation_date']).dt.year
    
    # Create a temporary dataframe with annual data
    temp_df = pd.DataFrame()
    for year in population_df['Year'].unique():
        year_pop = population_df[population_df['Year'] == year]['DCPOP'].values[0]
        for month in range(1, 13):
            temp_df = pd.concat([temp_df, pd.DataFrame({
                'Year': [year],
                'Month': [month],
                'Date': [pd.to_datetime(f"{year}-{month:02d}-01")],
                'Population': [year_pop]
            })])
    
    # Sort by date
    temp_df = temp_df.sort_values('Date')
    
    # Interpolate to smooth out the steps
    temp_df['Population'] = temp_df['Population'].interpolate(method='cubic')
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        temp_df[['Year', 'Month', 'Population']],
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added Population data (interpolated to monthly)")
except Exception as e:
    print(f"Error processing Population data: {e}")

Added Population data (interpolated to monthly)


### Process Poverty Rate Data (annual)

In [40]:
try:
    poverty_df = pd.read_csv(os.path.join('Data', 'DC_Poverty_Rate.csv'))
    
    # Create a temporary dataframe with annual data
    temp_df = pd.DataFrame()
    for year in poverty_df['Year'].unique():
        year_poverty = poverty_df[poverty_df['Year'] == year]['Poverty Rate (%)'].values[0]
        for month in range(1, 13):
            temp_df = pd.concat([temp_df, pd.DataFrame({
                'Year': [year],
                'Month': [month],
                'Date': [pd.to_datetime(f"{year}-{month:02d}-01")],
                'Poverty_Rate': [year_poverty]
            })])
    
    # Sort by date
    temp_df = temp_df.sort_values('Date')
    
    # Interpolate to smooth out the steps
    temp_df['Poverty_Rate'] = temp_df['Poverty_Rate'].interpolate(method='cubic')
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        temp_df[['Year', 'Month', 'Poverty_Rate']],
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added Poverty Rate data (interpolated to monthly)")
except Exception as e:
    print(f"Error processing Poverty Rate data: {e}")

Added Poverty Rate data (interpolated to monthly)


### Process Marriage Rate Data

In [41]:
try:
    marriage_df = pd.read_csv(os.path.join('Data', 'DC_Marriage_Rate.csv'))
    
    # Convert from wide to long format
    years = marriage_df.columns.tolist()
    marriage_rates = marriage_df.iloc[0].tolist()
    
    # Create a proper dataframe
    marriage_annual = pd.DataFrame({
        'Year': [int(year) for year in years],
        'Marriage_Rate': marriage_rates
    })
    
    # Create a temporary dataframe with annual data
    temp_df = pd.DataFrame()
    for year in marriage_annual['Year'].unique():
        year_rate = marriage_annual[marriage_annual['Year'] == year]['Marriage_Rate'].values[0]
        for month in range(1, 13):
            temp_df = pd.concat([temp_df, pd.DataFrame({
                'Year': [year],
                'Month': [month],
                'Date': [pd.to_datetime(f"{year}-{month:02d}-01")],
                'Marriage_Rate': [year_rate]
            })])
    
    # Sort by date
    temp_df = temp_df.sort_values('Date')
    
    # Interpolate to smooth out the steps
    temp_df['Marriage_Rate'] = temp_df['Marriage_Rate'].interpolate(method='cubic')
    
    # Apply seasonal pattern (more marriages in summer)
    seasonal_pattern = [0.8, 0.8, 0.9, 1.0, 1.1, 1.3, 1.4, 1.2, 1.0, 0.9, 0.8, 0.8]
    
    # Calculate annual average to preserve
    yearly_avg = temp_df.groupby('Year')['Marriage_Rate'].mean().reset_index()
    
    # Apply seasonal pattern
    for year in temp_df['Year'].unique():
        # Get the annual average
        annual_avg = yearly_avg[yearly_avg['Year'] == year]['Marriage_Rate'].values[0]
        
        # Apply seasonal pattern to each month
        for month in range(1, 13):
            month_idx = ((temp_df['Year'] == year) & (temp_df['Month'] == month))
            seasonal_factor = seasonal_pattern[month-1]
            
            # Calculate the seasonally adjusted rate
            temp_df.loc[month_idx, 'Marriage_Rate'] = annual_avg * seasonal_factor
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        temp_df[['Year', 'Month', 'Marriage_Rate']],
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added Marriage Rate data (interpolated to monthly with seasonal pattern)")
except Exception as e:
    print(f"Error processing Marriage Rate data: {e}")
    import traceback
    traceback.print_exc()

Added Marriage Rate data (interpolated to monthly with seasonal pattern)


### Process Divorce Rate Data

In [42]:
try:
    divorce_df = pd.read_csv(os.path.join('Data', 'DC_Divorce_Rate.csv'))
    
    # Convert from wide to long format
    years = divorce_df.columns.tolist()
    divorce_rates = divorce_df.iloc[0].tolist()
    
    # Create a proper dataframe
    divorce_annual = pd.DataFrame({
        'Year': [int(year) for year in years],
        'Divorce_Rate': divorce_rates
    })
    
    # Create a temporary dataframe with annual data
    temp_df = pd.DataFrame()
    for year in divorce_annual['Year'].unique():
        year_rate = divorce_annual[divorce_annual['Year'] == year]['Divorce_Rate'].values[0]
        for month in range(1, 13):
            temp_df = pd.concat([temp_df, pd.DataFrame({
                'Year': [year],
                'Month': [month],
                'Date': [pd.to_datetime(f"{year}-{month:02d}-01")],
                'Divorce_Rate': [year_rate]
            })])
    
    # Sort by date
    temp_df = temp_df.sort_values('Date')
    
    # Interpolate to smooth out the steps
    temp_df['Divorce_Rate'] = temp_df['Divorce_Rate'].interpolate(method='cubic')
    
    # Apply seasonal pattern (less pronounced than marriage)
    seasonal_pattern = [1.05, 1.0, 1.0, 0.95, 0.95, 0.95, 1.0, 1.0, 1.05, 1.05, 1.0, 1.0]
    
    # Calculate annual average to preserve
    yearly_avg = temp_df.groupby('Year')['Divorce_Rate'].mean().reset_index()
    
    # Apply seasonal pattern
    for year in temp_df['Year'].unique():
        # Get the annual average
        annual_avg = yearly_avg[yearly_avg['Year'] == year]['Divorce_Rate'].values[0]
        
        # Apply seasonal pattern to each month
        for month in range(1, 13):
            month_idx = ((temp_df['Year'] == year) & (temp_df['Month'] == month))
            seasonal_factor = seasonal_pattern[month-1]
            
            # Calculate the seasonally adjusted rate
            temp_df.loc[month_idx, 'Divorce_Rate'] = annual_avg * seasonal_factor
    
    # Merge with monthly dataframe
    monthly_df = pd.merge(
        monthly_df,
        temp_df[['Year', 'Month', 'Divorce_Rate']],
        on=['Year', 'Month'],
        how='left'
    )
    
    print(f"Added Divorce Rate data (interpolated to monthly with seasonal pattern)")
except Exception as e:
    print(f"Error processing Divorce Rate data: {e}")

Added Divorce Rate data (interpolated to monthly with seasonal pattern)


## 4. Final Data Cleaning

In [45]:
# Add Date column for sorting and time series analysis
monthly_df['Date'] = pd.to_datetime(monthly_df['Year'].astype(str) + '-' + monthly_df['Month'].astype(str) + '-01')

# Sort by date
monthly_df = monthly_df.sort_values('Date')

# Handle missing values
for column in monthly_df.columns:
    if column not in ['Date', 'Year', 'Month']:
        # Forward fill missing values
        monthly_df[column] = monthly_df[column].fillna(method='ffill')
        
        # Backward fill for any remaining NAs at the beginning
        monthly_df[column] = monthly_df[column].fillna(method='bfill')

# Check for any remaining missing values
missing_values = monthly_df.isnull().sum()
print("\nMissing values after cleaning:")
print(missing_values)

# Save the processed DataFrame only to the root directory
monthly_df.to_csv('dc_economic_monthly_dataframe.csv', index=False)
print(f"\nSaved monthly dataframe to dc_economic_monthly_dataframe.csv in the root directory")


Missing values after cleaning:
Date                       0
Year                       0
Month                      0
Unemployment_Rate          0
CPI                        0
Interest_Rate              0
Mortgage_Rate              0
ZHVI                       0
GDP_Growth                 0
Median_Household_Income    0
Population                 0
Poverty_Rate               0
Marriage_Rate              0
Divorce_Rate               0
dtype: int64

Saved monthly dataframe to dc_economic_monthly_dataframe.csv in the root directory


/tmp/ipykernel_6779/4103449650.py:11: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  monthly_df[column] = monthly_df[column].fillna(method='ffill')
/tmp/ipykernel_6779/4103449650.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  monthly_df[column] = monthly_df[column].fillna(method='bfill')
